In [3]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [4]:
#connect
drive.mount('/content/drive')
path = '/content/drive/MyDrive/test_opencv/project'
#show
os.listdir(path)

Mounted at /content/drive


[]

In [5]:
# 1. DATA
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/test_opencv/data_face_features.pickle", mode="rb"))

In [6]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.06073355  0.15911761 -0.11765669 ... -0.03886803  0.04120641
   -0.00426301]]

 [[ 0.03338404  0.20637566 -0.06423515 ... -0.05967721  0.06654365
    0.01773296]]

 [[ 0.05669597  0.2328304  -0.06432109 ... -0.04924931  0.11447396
    0.02399153]]

 ...

 [[ 0.02085523  0.22127947 -0.06984795 ...  0.02055509  0.0865813
    0.10365864]]

 [[ 0.02926855  0.21446565 -0.03790818 ... -0.06640885  0.01369472
    0.06734287]]

 [[-0.01456819  0.00786272  0.08517674 ...  0.11353574  0.04404245
    0.12897009]]]
['172100257_LuongThanhPhong' '172100257_LuongThanhPhong'
 '172100257_LuongThanhPhong' ... '151901205_BuiKhanhBinh'
 '151901205_BuiKhanhBinh' '151901205_BuiKhanhBinh']


In [7]:
x.shape

(4507, 1, 128)

In [8]:
y.shape

(4507,)

In [9]:
x = x.reshape(-1, 128)

In [10]:
x.shape

(4507, 128)

In [11]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)

In [12]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3605, 128), (902, 128), (3605,), (902,))

In [13]:
# 2. Train machine learning

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [15]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [16]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [17]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.71
Accurency Test = 0.70
F1 Score Train = 0.51
F1 Score Test = 0.48


In [18]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [19]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.85
F1 Score Train = 0.83
F1 Score Test = 0.75


In [20]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [21]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.75
F1 Score Train = 1.00
F1 Score Test = 0.66


In [22]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [23]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [24]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.85
F1 Score Train = 0.97
F1 Score Test = 0.76


In [25]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [26]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   5.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   5.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   5.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   5.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   4.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gam

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [27]:
model_best_estimator = model_grid.best_estimator_

In [28]:
model_grid.best_score_

0.8357820692037464

In [29]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/test_opencv/ml_face_person_identity.pkl", mode='wb'))